In [4]:
from evaluator import *

In [5]:
FILE = 'data/imputed_ant_phi_100.csv'    # Dataset to evalulate

In [6]:
# Create calculators using the data
calculators: dict[str, DistanceCalculator] = {
    'syntactic': create_syntactic_calculator(FILE),
    'morphological': create_morphological_calculator(FILE),
    'inventory': create_inventory_calculator(FILE),
    'phonological': create_phonological_calculator(FILE),
    'featural': create_featural_calculator(FILE)
}

# Define features to be used in the evaluation
features: list[str] = [
    'syntactic', 
    'inventory', 
    'phonological',
    'featural',
    'morphological'
]

# Define tasks with their evaluation metrics and whether they need to be converted from ISO code to glottocode
tasks: dict[str, tuple[str, bool]] = {
    'mt': ('BLEU', True),
    'dep': ('accuracy', True),
    'el': ('accuracy', True),
    'pos': ('accuracy', True),
    'taxi1500': ('f1_score', False),
}

In [7]:
# Create the evaluator with the calculators
evaluator = LangRankEvaluator(
    calculators = calculators,
    iso_map_file = 'data/code_mapping.csv'
)

In [8]:
# 
for task in tasks:
    df = evaluator.replace_distances(
        dataset_path = f'data/{task}.csv', # path to the task dataset (the one containing columns for task lang, transfer lang, performance)
        distance_types = features, # list of distance types to replace in the dataset. these should match the keys of the dict passed into the evaluator.
        iso_conversion = tasks[task][1] # indicate whether to convert lang codes in your dataset from ISO to Glottocode using the file in self.iso_map_file
    )
    # run LangRank and evaluate task performance
    score = evaluator.evaluate(
        data = df,
        features = features + ['geographic', 'genetic'], # list of columns in the dataset CSV to use for evaluation
        performance_col_name = tasks[task][0], # name of the column in the dataset CSV containing task performance scores
    )
    print(f'Task: {task} NDCG: {score}')

Task: mt NDCG: 30.721582366789193
Task: dep NDCG: 73.41818968516507
Task: el NDCG: 60.35768790909402
Task: pos NDCG: 16.58458584715933
Task: taxi1500 NDCG: 29.293423821714203
